# Calculate 4D features for reactants
## Note: run in `crest` kernel

Here, we calculate steric and electronic descriptors for conformer ensembles of reactants. 
We use [CREST](https://crest-lab.github.io/crest-docs/) to generate conformer ensembles and [MORFEUS](https://digital-chemistry-laboratory.github.io/morfeus/index.html) to calculate Boltzmann-weighted descriptors.

In [ ]:
import sys
import pathlib
import json
sys.path.append(str(pathlib.Path().absolute().parent))

from rdkit import Chem, RDLogger
from rdkit.Chem import Draw, AllChem, rdmolfiles, SaltRemover, MolStandardize
from rdkit.Chem.rdmolfiles import MolToXYZFile
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.Chem.MolStandardize import rdMolStandardize
RDLogger.DisableLog('rdApp.info')
import pandas as pd
import morfeus
from morfeus import SASA, XTB, Sterimol
from morfeus.conformer import ConformerEnsemble, conformers_from_rdkit
import numpy as np
import pickle as pkl
import xtb  # n.b. we import this only b/c MORFEUS has an internal error where it can fail to resolve the xtb.utils module if we don't import this

from src.util.definitions import DATA_ROOT
from src.util.rdkit_util import standardize_building_block

In [ ]:
data = pd.read_csv(DATA_ROOT / "synferm_dataset_2024-04-18_38586records.csv")

def rename_func(s):
    return s.split("_")[-1]
building_blocks = pd.concat([data[[f"{s}_long", f"{s}_smiles"]].drop_duplicates().rename(columns=rename_func).assign(bb_type=s) for s in "IMT"])
assert len(building_blocks) == 66 + 70 + 41
building_blocks.head()

In [ ]:
# define some utility functions
def standardise_smiles(smiles):
    """Desalt and neutralize (except KAT B- that cannot be neutralized). Return the standardized SMILES"""
    remover = SaltRemover()
    uncharger = rdMolStandardize.Uncharger()
    mol = Chem.MolFromSmiles(smiles)
    res = remover.StripMol(mol)
    # remove counterion
    largest_Fragment = rdMolStandardize.LargestFragmentChooser()
    largest_mol = largest_Fragment.choose(mol)
    # neutralize monomers and terminators (i.e. ammoniums) the Uncharger will leave the [B-] as is.
    return Chem.MolToSmiles(uncharger.uncharge(largest_mol))

def smiles_to_xyz(smiles, output_filename):
    """A function that generates a xyz file for a SMILES input"""
    # generate Mol
    mol = smiles_to_mol(smiles)

    # Write the molecule to an XYZ file
    MolToXYZFile(mol, output_filename)
    # Write charge to .CHRG file for xtb
    with open(output_filename.parent / ".CHRG", "w") as f:
        f.write(f"{Chem.GetFormalCharge(mol)}\n")

    return None

def smiles_to_mol(smiles):
    """
    A function that generates a rdkit Mol for a SMILES input, 
    adding explicit Hs and optimizing geometry with MMFF94.
    """
    # Convert the SMILES string to a molecule object
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)  # Add hydrogens

    # Generate 3D coordinates
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    AllChem.MMFFOptimizeMolecule(mol)  # Optimize the geometry

    return mol

In [ ]:
# Update the SMILES column which contains the standardized smiles
building_blocks['smiles_standard'] = building_blocks['smiles'].apply(standardise_smiles)

In [ ]:
# Make directory structure for CREST calculations
calc_dir = DATA_ROOT / "feature_calculations"
calc_dir.mkdir(exist_ok=True)

for long in building_blocks["long"]:
    (calc_dir / long).mkdir(exist_ok=True)

In [ ]:
# export XYZ files
for i, row in building_blocks.iterrows():
    smiles_to_xyz(row["smiles_standard"], calc_dir / row["long"] / "mmff94_out.xyz")

In [ ]:
building_blocks["mol"] = building_blocks["smiles_standard"].apply(lambda x: smiles_to_mol(x))
building_blocks

## External optimization
Structures were optimized on the cluster. Each structure was pre-optimized at the xtb-gnf2 level. Conformers were generated using crest and optimized at the xtb-gnf level, with singlepoint calculations at the xtv-gnf2 level.

Command to run on the cluster:

```bash
sbatch -n 1 --cpus-per-task=1 --time=4:00:00 --mem-per-cpu=2048 --output="stdout.txt" --error="stderr.txt" --open-mode=append --wrap="crest xtbopt.xyz --gbsa h2o -T 1 --gfn2//gfnff"
```

Optimized conformer ensembles are in `$PROJECT_ROOT/data/crest_opt/<building_block>/`.

In [ ]:
ensemble_dir = DATA_ROOT / "crest_opt"

In [ ]:
def ce_from_crest(crest_dir, mol):
    """
    Creates and refines a ConformerEnsemble from CREST output folder.
    Problem with the CREST outputs is that they do not have any connectivity information (.xyz files)
    We can assign the original RDKit Mols. After all CREST should conserve atom order.
    
    Args:
        crest_dir (str): Directory containing CREST output.
        mol (Chem.Mol): RDKit Mol that the input to crest was generated from.

    Returns:
        A ConformerEnsemble: Refined ensemble of conformers after sorting, adding RDKit Mol,
        and pruning based on RMSD and energy.
    """
    
    # early exit if crest did not converge
    if (crest_dir / "NOT_CONVERGED").exists():
        print(f"Skipping {crest_dir.name}. CREST did not converge.")
        return None
    
    # Generate MORFEUS ConformerEnsemble object from CREST folder and sort it energetically
    ce = ConformerEnsemble.from_crest(crest_dir)
    ce.sort()

    # Add molecule representation
    ce.mol = mol
    
    # quick check: is the atom order identical?
    elem = [a.GetAtomicNum() for a in mol.GetAtoms()]
    all(elem == ce.elements)
    
    # Obtain connectivity matrix and charges from mol
    (       elements,
            conformer_coordinates,
            energies,
            connectivity_matrix,
            charges,
            _,
        ) = conformers_from_rdkit(mol)
    ce.connectivity_matrix = connectivity_matrix
    ce.formal_charges = charges
    ce.charge = int(charges.sum())

    # Prune according to rmsd and energy
    ce.prune_rmsd()
    ce.prune_energy()

    return ce

In [ ]:
# import the externally computed CREST conformer ensembles
building_blocks["ce"] = building_blocks.apply(lambda row: ce_from_crest(ensemble_dir / row["long"], row["mol"]), axis=1)
building_blocks.head()

## Feature calculations
We will calculate the following features using [MORFEUS](https://digital-chemistry-laboratory.github.io/morfeus/index.html):
- HOMO/LUMO energy
- Dipole of the molecule
- I only:
    - Sterimol params L, B1, B5 for the C_KAT - C_alpha bond
    - SASA (solvent accessible surface area) for C_KAT
- M only:
    - Sterimol params L, B1, B5 for the N - C_beta3 bond
    - Sterimol params L, B1, B5 for the O - C_beta2 bond
    - SASA (solvent accessible surface area) for N
- TerTH only:
    - Sterimol params L, B1, B5 for the C_carbonyl - C_alpha bond
- TerABT only:
    - Sterimol params L, B1, B5 for the N - C_arom bond
    - Sterimol params L, B1, B5 for the S - C_arom bond
    
Note on Sterimol params: For ligands these use the dummy atom (H) bond with the first atom of the substituent (e.g. for an NHC that would be the 2-carbon atom of the imidazolidine). We want to determine steric parameters of the "substituents" attached to our reactive groups, so for KATs for example, we use the KAT carbon as dummy atom and the alpha carbon as the "first atom" of the substituent.

Note on terminators: TH features will be zero-padded to match lenght of ABT features. An additional binary feature is introduced to distinguish ABT and TH terminators. It will be 0 for ABTs and 1 for THs.

In [ ]:
def get_dipole_homo_lumo(ce):
    """Calculate electronic properties with XTB. Return Boltzmann-weighted dipole, HOMO, and LUMO."""
    charge = ce.charge
    for conformer in ce:
        xtb_calc = XTB(conformer.elements, conformer.coordinates, charge=charge, solvent="h2o")
        dipole = xtb_calc.get_dipole()
        conformer.properties["dipole"] = np.linalg.norm(dipole)
        conformer.properties["homo"] = xtb_calc.get_homo()
        conformer.properties["lumo"] = xtb_calc.get_lumo()
    return ce.boltzmann_statistic("dipole"), ce.boltzmann_statistic("homo"), ce.boltzmann_statistic("lumo")


In [ ]:
# Generate new columns with molecule dipole, HOMO, LUMO
building_blocks[["dipole", "homo", "lumo"]] = building_blocks[["ce"]].apply(lambda x: get_dipole_homo_lumo(x["ce"]), axis=1, result_type="expand")

In [ ]:
# save just to be sure
with open(DATA_ROOT / "dataframe_tmp.pkl", "wb") as f:
    pkl.dump(building_blocks, f)

In [ ]:
# load from dump
with open(DATA_ROOT / "dataframe_tmp.pkl", "rb") as f:
    building_blocks = pkl.load(f)

In [ ]:
building_blocks

For calculations that are specific to the building block type, we need to split the DataFrame

In [ ]:
# split building blocks into types
initiators = building_blocks.loc[building_blocks["bb_type"] == "I"].copy()
monomers = building_blocks.loc[building_blocks["bb_type"] == "M"].copy()
terminators_abt = building_blocks.loc[building_blocks["long"].str.startswith("TerABT")].copy()
terminators_th = building_blocks.loc[building_blocks["long"].str.startswith("TerTH")].copy()

In [ ]:
# find atom_indices for all building block classes
PATTERN_I = Chem.MolFromSmarts("[#6]-[CX3H0](=[OX1])-[$([B-](-F)(-F)-F)]")  # match KAT group in order C_alpha, C_KAT, O, B
PATTERN_M = Chem.MolFromSmarts("C-1-C-[$(C-2-C(=O)-O-C-O-2)]-[OX2]-[NX3H1]-1")  # match isoxazolidine in order C_b3, C_b2, C_quart, O, N
PATTERN_TERTH = Chem.MolFromSmarts("[#6]-[CX3](=[SX1])-[NX3H1]-[NX3H2]")  # match thiohydrazide in order C_alpha, C_carbonyl, S, NH, NH2
PATTERN_TERABT = Chem.MolFromSmarts("[SH1]-c:1:c(-[NH2]):c:c:c:c:1")  # match ABT in order S, C_S, C_N, N, four benzene carbons

def get_KAT_idx_from_ce(ce, has_Hs = True):
    """
    Get atom indices for the KAT group in an initiator (or other molecule).

    Args:
        ce: MORFEUS ConformerEnsemble object containing only one KAT group
        has_Hs: whether or not the molecule has explicit hydrogens. Defaults to True.

    Returns:
        Atom indices of C_alpha, C_KAT, O, B
    """

    # Get RDKit Mol object with explicit hydrogens
    mol = ce.mol
    if not has_Hs:
        mol = Chem.AddHs(mol)

    # Search the substructure
    C_alpha, C_kat, O, B = mol.GetSubstructMatch(PATTERN_I)
    
    # Return the relevant atom indices, + 1 to account for MORFEUS numbering
    return C_alpha + 1, C_kat + 1, O + 1, B + 1

def get_isoxazolidine_idx_from_ce(ce, has_Hs = True):
    """
    Get atom indices for the isoxazolidine ring in a monomer (or comparable molecule).

    Args:
        ce: MORFEUS ConformerEnsemble object containing only one isoxazolidine group
        has_Hs: whether or not the molecule has explicit hydrogens. Defaults to True.

    Returns:
        Atom indices of C_b3, C_b2, C_quart, O, N
    """

    # Get RDKit Mol object with explicit hydrogens
    mol = ce.mol
    if not has_Hs:
        mol = Chem.AddHs(mol)

    # Search the substructure
    C_b3, C_b2, C_quart, O, N = mol.GetSubstructMatch(PATTERN_M)
    
    # Return the relevant atom indices, + 1 to account for MORFEUS numbering
    return C_b3 + 1, C_b2 + 1, C_quart + 1, O + 1, N + 1

def get_thiohydrazide_idx_from_ce(ce, has_Hs = True):
    """
    Get atom indices for the thiohydrazide group in a terminator (or comparable molecule).

    Args:
        ce: MORFEUS ConformerEnsemble object containing only one thiohydrazide group
        has_Hs: whether or not the molecule has explicit hydrogens. Defaults to True.

    Returns:
        Atom indices of C_alpha, C_carbonyl, S, NH, NH2
    """

    # Get RDKit Mol object with explicit hydrogens
    mol = ce.mol
    if not has_Hs:
        mol = Chem.AddHs(mol)

    # Search the substructure
    C_alpha, C_carbonyl, S, NH, NH2 = mol.GetSubstructMatch(PATTERN_TERTH)
    
    # Return the relevant atom indices, + 1 to account for MORFEUS numbering
    return C_alpha + 1, C_carbonyl + 1, S + 1, NH + 1, NH2 + 1

def get_aminobenzenethiol_idx_from_ce(ce, has_Hs = True):
    """
    Get atom indices for the aminobenzenethiol group in a terminator (or comparable molecule).

    Args:
        ce: MORFEUS ConformerEnsemble object containing only one aminobenzenethiol group
        has_Hs: whether or not the molecule has explicit hydrogens. Defaults to True.

    Returns:
        Atom indices of S, C_S, C_N, N
    """

    # Get RDKit Mol object with explicit hydrogens
    mol = ce.mol
    if not has_Hs:
        mol = Chem.AddHs(mol)

    # Search the substructure
    S, C_S, C_N, N, _, _, _, _ = mol.GetSubstructMatch(PATTERN_TERABT)  # we discard the 4 benzene C indices
    
    # Return the relevant atom indices, + 1 to account for MORFEUS numbering
    return S + 1, C_S + 1, C_N + 1, N + 1


In [ ]:
# Generate new columns for relevant atom indices for initiators
(
    initiators['C_alpha_idx'], 
    initiators['C_kat_idx'], 
    initiators['O_idx'], 
    initiators['B_idx']
) = zip(*initiators['ce'].apply(get_KAT_idx_from_ce))
initiators.head()

In [ ]:
# Generate new columns for relevant atom indices for monomers
(
    monomers['C_beta3_idx'],
    monomers['C_beta2_idx'],
    monomers['C_quart_idx'],
    monomers['O_idx'],
    monomers['N_idx']
) = zip(*monomers['ce'].apply(get_isoxazolidine_idx_from_ce))


In [ ]:
# Generate new columns for relevant atom indices for terminators_th
(
    terminators_th['C_alpha_idx'],
    terminators_th['C_carbonyl_idx'],
    terminators_th['S_idx'],
    terminators_th['NH_idx'],
    terminators_th['NH2_idx']
) = zip(*terminators_th['ce'].apply(get_thiohydrazide_idx_from_ce))


In [ ]:
# Generate new columns for relevant atom indices for terminators_abt
(
    terminators_abt['S_idx'],
    terminators_abt['C_S_idx'],
    terminators_abt['C_N_idx'],
    terminators_abt['N_idx']
) = zip(*terminators_abt['ce'].apply(get_aminobenzenethiol_idx_from_ce))


In [ ]:
# define functions to obtain steric properties

def get_bond_sterimol(ce, atom_idx_1, atom_idx_2):
    """
    Function to get Sterimol parameters for the specified bond.
    
    Args:
        ce: Conformer Ensemble from MORFEUS
        atom_idx_1: Idx of the dummy atom (in MORFEUS counting style, i.e. start at 1)
        atom_idx_2: Idx of the first atom in the substituent (in MORFEUS counting style)    
    Returns:
        tuple: Boltzmann-weighted B1, B5, and L parameters for the conformer ensemble
    """
    for conformer in ce:
        sterimol = Sterimol(conformer.elements, conformer.coordinates, atom_idx_1, atom_idx_2)
        conformer.properties[f"B1"] = sterimol.B_1_value
        conformer.properties[f"B5"] = sterimol.B_5_value
        conformer.properties[f"L"] = sterimol.L_value

    return ce.boltzmann_statistic(f"B1"), ce.boltzmann_statistic(f"B5"), ce.boltzmann_statistic(f"L")

def get_atom_sasa(ce, atom_idx):
    """Get solvent accessible surface area (SASA) for the specified atom"""
    for conformer in ce:
        sasa = SASA(conformer.elements, conformer.coordinates)
        conformer.properties["SASA"] = sasa.atom_areas[atom_idx]
    return ce.boltzmann_statistic("SASA")


In [ ]:
# Generate new column with Sterimol parameters of C_kat - C_alpha bond
Ls, B1s, B5s = [], [], []
for index, row in initiators.iterrows():
    B1, B5, L = get_bond_sterimol(row['ce'], row['C_kat_idx'], row['C_alpha_idx'])
    B1s.append(B1)
    B5s.append(B5)
    Ls.append(L)
initiators['B1'] = B1s
initiators['B5'] = B5s
initiators['L'] = Ls

In [ ]:
# Generate new column with SASA of KAT carbon atom
sasas = []
for index, row in initiators.iterrows():
    sasas.append(get_atom_sasa(row['ce'], row['C_kat_idx']))
initiators['SASA_C_KAT'] = sasas

In [ ]:
# Generate new column with Sterimol parameters of N - C_beta3 bond
Ls, B1s, B5s = [], [], []
for index, row in monomers.iterrows():
    B1, B5, L = get_bond_sterimol(row['ce'], row['N_idx'], row['C_beta3_idx'])
    B1s.append(B1)
    B5s.append(B5)
    Ls.append(L)
monomers['beta3_B1'] = B1s
monomers['beta3_B5'] = B5s
monomers['beta3_L'] = Ls

# Generate new column with Sterimol parameters of O - C_beta2 bond
Ls, B1s, B5s = [], [], []
for index, row in monomers.iterrows():
    B1, B5, L = get_bond_sterimol(row['ce'], row['O_idx'], row['C_beta2_idx'])
    B1s.append(B1)
    B5s.append(B5)
    Ls.append(L)
monomers['beta2_B1'] = B1s
monomers['beta2_B5'] = B5s
monomers['beta2_L'] = Ls

In [ ]:
# Generate new column with SASA of isoxazolidine nitrogen atom
sasas = []
for index, row in monomers.iterrows():
    sasas.append(get_atom_sasa(row['ce'], row['N_idx']))
monomers['SASA_N'] = sasas

In [ ]:
# Generate new column with Sterimol parameters of C_carbonyl - C_alpha bond
Ls, B1s, B5s = [], [], []
for index, row in terminators_th.iterrows():
    B1, B5, L = get_bond_sterimol(row['ce'], row['C_carbonyl_idx'], row['C_alpha_idx'])
    B1s.append(B1)
    B5s.append(B5)
    Ls.append(L)
terminators_th['B1'] = B1s
terminators_th['B5'] = B5s
terminators_th['L'] = Ls

In [ ]:
# Generate new column with Sterimol parameters of N - C_N bond
Ls, B1s, B5s = [], [], []
for index, row in terminators_abt.iterrows():
    B1, B5, L = get_bond_sterimol(row['ce'], row['N_idx'], row['C_N_idx'])
    B1s.append(B1)
    B5s.append(B5)
    Ls.append(L)
terminators_abt['N_B1'] = B1s
terminators_abt['N_B5'] = B5s
terminators_abt['N_L'] = Ls

# Generate new column with Sterimol parameters of S - C_S bond
Ls, B1s, B5s = [], [], []
for index, row in terminators_abt.iterrows():
    B1, B5, L = get_bond_sterimol(row['ce'], row['S_idx'], row['C_S_idx'])
    B1s.append(B1)
    B5s.append(B5)
    Ls.append(L)
terminators_abt['S_B1'] = B1s
terminators_abt['S_B5'] = B5s
terminators_abt['S_L'] = Ls

In [ ]:
# pickle current state in case we need to add more things
with open(DATA_ROOT / "4D_features_backup.pkl", "wb") as f:
    pkl.dump((building_blocks, initiators, monomers, terminators_th, terminators_abt), f)

In [ ]:
# zero pad THs
terminators_th['S_B1'] = 0
terminators_th['S_B5'] = 0
terminators_th['S_L'] = 0

# encode T type
terminators_th['T_type'] = 1
terminators_abt['T_type'] = 0

In [ ]:
# get dictionary with features as np arrays
initiator_props = dict(zip(initiators["long"].to_numpy(), initiators[["dipole", "homo", "lumo", "B1", "B5", "L", "SASA_C_KAT"]].to_numpy()))
monomer_props = dict(zip(monomers["long"].to_numpy(), monomers[["dipole", "homo", "lumo", "beta3_B1", "beta3_B5", "beta3_L", "beta2_B1", "beta2_B5", "beta2_L", "SASA_N"]].to_numpy()))
terminator_th_props = dict(zip(terminators_th["long"].to_numpy(), terminators_th[["dipole", "homo", "lumo", "B1", "B5", "L", "S_B1", "S_B5", "S_L", "T_type"]].to_numpy()))
terminator_abt_props = dict(zip(terminators_abt["long"].to_numpy(), terminators_abt[["dipole", "homo", "lumo", "N_B1", "N_B5", "N_L", "S_B1", "S_B5", "S_L", "T_type"]].to_numpy()))

props = initiator_props | monomer_props | terminator_th_props | terminator_abt_props  # | combines dicts as of python3.9+

In [ ]:
# Write the dict to a JSON file
props_list = {key: value.tolist() for key, value in props.items()}  # convert arrays to lists for serialization
with open(DATA_ROOT / "4D_features.json", "w") as f:
    json.dump(props_list, f)

In [ ]:
# Write the same dict, but replace the compounds names with (canonical) SMILES
# (we will use this for the FromFileFeaturizer)
props_smiles = {building_blocks.loc[building_blocks["long"] == k, "smiles"].item(): v for k, v in props_list.items()}
# write to JSON
with open(DATA_ROOT / "4D_features_smiles.json", "w") as f:
    json.dump(props_smiles, f)

In [ ]:
# check that we have all building blocks that occur in our data

# standardize BBs in our data
bbs = []
bbs += data["I_smiles"].drop_duplicates().apply(standardize_building_block, return_smiles=True).to_list()
bbs += data["M_smiles"].drop_duplicates().apply(standardize_building_block, return_smiles=True).to_list()
bbs += data["T_smiles"].drop_duplicates().apply(standardize_building_block, return_smiles=True).to_list()

# standardize BBs in our property dict
standardized = [standardize_building_block(k, return_smiles=True) for k in props_smiles.keys()]

# this should be an empty set. Any SMILES occuring here, we do not have properties for
set(bbs).difference(set(standardized))

In [ ]:
# indicative only - these are SMILES we have properties for that do not occur in our data
set(standardized).difference(set(bbs))

In [ ]:
# Write properties to a CSV file that aligns with our data

features = []
for i, row in data.iterrows():
    features.append(np.concatenate((props[row["I_long"]], props[row["M_long"]], props[row["T_long"]])))
features

In [ ]:
# save to CSV
pd.concat(
    (data[["I_long", "M_long", "T_long"]], pd.DataFrame(features)),
    axis=1
).to_csv(
    DATA_ROOT / "synferm_dataset_2024-04-18_38586records_4Dfeatures.csv"
)